## Connect to database

In [1]:
# db 연결
import pymysql
mydb = pymysql.connect(
    host='localhost',
    port=3307,
    user='root',
    passwd='airscope',
    db='flight',
    charset='utf8',
    autocommit=True)

In [2]:
import import_ipynb
import pandas as pd
from spelling_corrector import _correction
import _get_intent_LSTM
import _get_tag
import _get_view

importing Jupyter notebook from spelling_corrector.ipynb
importing Jupyter notebook from _get_intent_LSTM.ipynb


Using TensorFlow backend.
c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


importing Jupyter notebook from _get_tag.ipynb


W1214 00:45:58.685973  6432 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1214 00:46:00.136977  6432 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1214 00:46:00.629597  6432 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1214 00:46:01.804871  6432 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name 

importing Jupyter notebook from _todateformat.ipynb
importing Jupyter notebook from _get_view.ipynb


In [3]:
print("the price is won-based")

the price is won-based


## get user input and do a spell check

In [29]:
user_sent = "sohow me the cheaapest flgidt from incheno to houston with a stopover in vancouver on december 14th"
user_sent = "show me the flight below 5000000 from incheon to tokyo departing today"
user_sent = "show me the flight from incheon to seattle on december 14th by asiana airline"
user_input = _correction(user_sent)
user_input

'show me the flight from incheon to seattle on december 14th by asiana airline'

# predict user intent (Intent classify)

In [17]:
intent = _get_intent_LSTM.get_intent(user_input)
intent = ''.join(intent)
print(intent)

AskFlight, AskFlightWithAirline


In [18]:
intent

'AskFlight, AskFlightWithAirline'

In [30]:
df = _get_tag.todf(user_input)
df

,Word,Prediction
0,show,O
1,me,O
2,the,O
3,flight,O
4,from,O
5,incheon,B-fromloc
6,to,O
7,seattle,B-toloc
8,on,O
9,december,B-depart_date.month_name


# get key elements (NER extract)

In [9]:
tagged = _get_tag.get_element(user_input, intent)
element = tagged

In [10]:
#tagged[tagged['tag'].isin(['dpttime'])].element.tolist()[0]==''
element

,tag,element
0,airline,
1,fromloc,incheon
2,stoploc,
3,toloc,jeju
4,dpttime,2019-12-05
5,arrtime,
6,cost,
7,cheapest,0


# get query result from the database

In [11]:
from _get_view import getroute

In [12]:
#결과 df route에 받아오기
route = getroute(intent, element, mydb)
route

here are your results: 
  airline     airline_name                       src_name src_city  \
0      OZ  Asiana Airlines  Incheon International Airport  Incheon   

   src_country                    dst_name dst_city  dst_country  stops  \
0  South Korea  Jeju International Airport     Jeju  South Korea      2   

             dpt_time            arr_time    cost  
0 2019-12-05 05:40:00 2019-12-05 12:45:00  389500  
stop info:
              name     city country  stop_order
0  Fukuoka Airport  Fukuoka   Japan           1
                           name   city      country  stop_order
0  Gimhae International Airport  Busan  South Korea           2


# when user types in additional input -> ellipsis function

In [13]:
second_input = "how about to japan"
#second_input = _correction(second_input)
new_sent, element = _get_tag.ellipsis(tagged, user_input, second_input)
element

,tag,element
0,airline,
1,fromloc,incheon
2,stoploc,
3,toloc,japan
4,dpttime,2019-12-05
5,arrtime,
6,cost,
7,cheapest,0


In [15]:
route2 = getroute(intent, element, mydb)
route2

here are your results: 
Oops! The flight you are looking for does not exist.
